In [1]:
%cd ~/TOA_causal

/home/jovyan/TOA_causal


In [ ]:
from synthetic.createdata import create_trainatestdata
create_trainatestdata('../data/')

Obtaining data for training...


/opt/conda/lib/python3.7/site-packages/skimage/external/tifffile/tifffile.py:2618: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
  8%|▊         | 1312/16384 [00:00<00:01, 13110.79it/s]

Creating Forward Model-based Matrix without position uncertainty
Creating SIR Matrix...


100%|██████████| 16384/16384 [00:01<00:00, 12941.76it/s]


Creating PA Matrix...
Applying Time Derivative Operator...


...

Creating PA Matrix...
Applying Time Derivative Operator...
Batch:  4   Sinogram:  3  de  2216   Image:  1  SNR(dB):  19
Batch:  4   Sinogram:  7  de  2216   Image:  2  SNR(dB):  20
Batch:  4   Sinogram:  11  de  2216   Image:  3  SNR(dB):  16
Batch:  4   Sinogram:  15  de  2216   Image:  4  SNR(dB):  31
Batch:  4   Sinogram:  19  de  2216   Image:  5  SNR(dB):  17
Batch:  4   Sinogram:  23  de  2216   Image:  6  SNR(dB):  25
Batch:  4   Sinogram:  27  de  2216   Image:  7  SNR(dB):  17
Batch:  4   Sinogram:  31  de  2216   Image:  8  SNR(dB):  13
Batch:  4   Sinogram:  35  de  2216   Image:  9  SNR(dB):  8
Batch:  4   Sinogram:  39  de  2216   Image:  10  SNR(dB):  39
Batch:  4   Sinogram:  43  de  2216   Image:  11  SNR(dB):  26
Batch:  4   Sinogram:  47  de  2216   Image:  12  SNR(dB):  14
Batch:  4   Sinogram:  51  de  2216   Image:  13  SNR(dB):  15
Batch:  4   Sinogram:  55  de  2216   Image:  14  SNR(dB):  14
Batch:  4   Sinogram:  59  de  2216   Image:  15  SNR(dB):  25
Batch:  

  8%|▊         | 1230/16384 [00:00<00:01, 12281.89it/s]

Creating Forward Model-based Matrix with position uncertainty
Creating SIR Matrix...


100%|██████████| 16384/16384 [00:01<00:00, 12689.29it/s]


Creating PA Matrix...
Applying Time Derivative Operator...
Batch:  5   Sinogram:  3  de  2216   Image:  1  SNR(dB):  12
Batch:  5   Sinogram:  7  de  2216   Image:  2  SNR(dB):  23
Batch:  5   Sinogram:  11  de  2216   Image:  3  SNR(dB):  17
Batch:  5   Sinogram:  15  de  2216   Image:  4  SNR(dB):  28
Batch:  5   Sinogram:  19  de  2216   Image:  5  SNR(dB):  34
Batch:  5   Sinogram:  23  de  2216   Image:  6  SNR(dB):  21
Batch:  5   Sinogram:  27  de  2216   Image:  7  SNR(dB):  46
Batch:  5   Sinogram:  31  de  2216   Image:  8  SNR(dB):  28
Batch:  5   Sinogram:  35  de  2216   Image:  9  SNR(dB):  13
Batch:  5   Sinogram:  39  de  2216   Image:  10  SNR(dB):  12
Batch:  5   Sinogram:  43  de  2216   Image:  11  SNR(dB):  32
Batch:  5   Sinogram:  47  de  2216   Image:  12  SNR(dB):  17
Batch:  5   Sinogram:  51  de  2216   Image:  13  SNR(dB):  38
Batch:  5   Sinogram:  55  de  2216   Image:  14  SNR(dB):  15
Batch:  5   Sinogram:  59  de  2216   Image:  15  SNR(dB):  49
Batch: 

In [3]:
import torch
import os
from tqdm import tqdm
import numpy as np
from TOA.mbfdunetln import MBPFDUNet
#from ANDMask.adam_flexible_weight_decay import AdamFlexibleWeightDecay
from torch.optim.lr_scheduler import MultiStepLR
from TOA.train import createForwMat
from utils.causal_utils import train,validation,testing,computing_metrics,load_traindataset,load_testdataset,load_ckp
from utils.noncausal_utils import load_traindataset_nc,train_nc

In [4]:
val_percent = 440.0/2216.0 # 440 para validacion, 1776 para train
le = 5
epochs = 50
cache_dir = '../data/'
fecha = '050822_17'
continue_training = False

In [5]:
if torch.cuda.is_available():
	device = torch.device("cuda")
else:
	device = torch.device("cpu")
print(f"Device to be used: {device}")


##Loss
loss_fn = torch.nn.MSELoss()

##TOA matrix
Ao = createForwMat()
Ao = torch.as_tensor(Ao).type(torch.float32)
Ao = Ao.to(device=device)

##Files
ckp_last = cache_dir + 'mbfdunetln' + fecha + '.pth' # name of the file of the saved weights of the trained net
ckp_best = cache_dir + 'mbfdunetln_best' + fecha + '.pth'

#if continue_training:
#	model, optimizer, epoch0, valid_loss_min = load_ckp(ckp_last, model, optimizer)
#    lr_scheduler = MultiStepLR(optimizer,milestones=[le * epochs * 3 // 4],gamma=0.1,last_epoch = epoch0 - 1)
#else:
	#epoch0 = 0
	#valid_loss_min = np.inf

  8%|▊         | 1229/16384 [00:00<00:01, 12286.15it/s]

Device to be used: cuda
Creating Forward Model-based Matrix without position uncertainty
Creating SIR Matrix...


100%|██████████| 16384/16384 [00:01<00:00, 12909.21it/s]


Creating PA Matrix...
Applying Time Derivative Operator...
done
done


In [ ]:
alphas = [1e-4,5e-4,1e-3]
bs = [1,2,3] # per environment
taus = [0.4,0.8]

checkpoint = {'valid_loss_min': np.inf}

epoch0 = 0
for batchsize in bs:
    train_loaders, val_loader = load_traindataset(cache_dir,val_percent,batchsize,val_batchsize=40,le = le)
    for lr in alphas:
        for agreement_threshold in taus:
            model = MBPFDUNet().to(device=device)
            checkpoint['state_dict'] = model.state_dict()
            optimizer = torch.optim.Adam(model.parameters(),lr=lr)
            #optimizer = AdamFlexibleWeightDecay(model.parameters(),lr=lr,weight_decay_order='before',weight_decay=wd)
            checkpoint['learning_rate'] = lr
            checkpoint['batchsize'] = batchsize
            checkpoint['agreement_threshold'] = agreement_threshold
            checkpoint['optimizer'] = optimizer.state_dict()
            checkpoint['epoch'] = epoch0
            lr_scheduler = MultiStepLR(optimizer,milestones=[le * epochs * 3 // 4],gamma=0.1)
            for epoch in tqdm(range(epoch0 + 1, epochs + 1)):
                train(model,device,train_loaders,optimizer,n_agreement_envs=le,Ao=Ao,loss_fn=loss_fn,agreement_threshold=agreement_threshold,scheduler=lr_scheduler)
                checkpoint['epoch'] = epoch
                checkpoint['valid_loss_min'] = validation(model, device, val_loader, optimizer, loss_fn, Ao, checkpoint, ckp_last, ckp_best)

 50%|█████     | 25/50 [1:48:06<1:48:38, 260.74s/it]

In [ ]:
model, optimizer, best_epoch, valid_loss_min, best_lr, best_bs, best_threshold = load_ckp(ckp_best, model, optimizer)

In [ ]:
#from importlib import reload 
#import utils.noncausal_utils 
#reload(utils.noncausal_utils)
#from utils.noncausal_utils import train_nc

In [ ]:
ckp_benchmark_last = cache_dir + 'benchmark' + fecha + '.pth'
ckp_benchmark_best = cache_dir + 'benchmark_best' + fecha + '.pth'
checkpoint_nc = {'valid_loss_min': np.inf, 'agreement_threshold' : 0.0}
epoch0 = 0
for batchsize in bs:
    train_loader_nc, val_loader_nc = load_traindataset_nc(cache_dir,val_percent,batchsize*le,val_batchsize=40,le = le)
    for lr in alphas:
        model_nc = MBPFDUNet().to(device=device)
        checkpoint_nc['state_dict'] = model_nc.state_dict()
        optimizer_nc = torch.optim.Adam(model_nc.parameters(),lr=lr)
        checkpoint_nc['learning_rate'] = lr
        checkpoint_nc['batchsize'] = batchsize
        checkpoint_nc['optimizer'] = optimizer_nc.state_dict()
        checkpoint_nc['epoch'] = epoch0
        lr_scheduler_nc = MultiStepLR(optimizer_nc,milestones=[le * epochs * 3 // 4],gamma=0.1)
        for epoch in tqdm(range(epoch0 + 1, epochs + 1)):
            train_nc(model_nc,device,train_loader_nc,optimizer_nc,Ao=Ao,loss_fn=loss_fn,scheduler=lr_scheduler_nc)
            checkpoint_nc['epoch'] = epoch
            checkpoint_nc['valid_loss_min'] = validation(model_nc, device, val_loader_nc, optimizer_nc, loss_fn, Ao, checkpoint_nc, ckp_benchmark_last, ckp_benchmark_best)

In [ ]:
model_nc, optimizer_nc, best_epoch_nc, vlm_nc, lr_nc, bs_nc, at_nc= load_ckp(ckp_benchmark_best, model_nc, optimizer_nc)

In [ ]:
test_loaders = load_testdataset(cache_dir)
le_test = len(test_loaders)
SSIM = [[] for _ in range(le_test)] 
PC = [[] for _ in range(le_test)] 
RMSE = [[] for _ in range(le_test)]
PSNR = [[] for _ in range(le_test)]
for j in range(le_test):
    iterator = iter(test_loaders[j])
    while 1:
        try:
            data_test = next(iterator)
        except StopIteration:
            break  
        a,b,c,d=computing_metrics(data_test[0].to("cpu"),data_test[1].to("cpu"),Ao.to(device="cpu"),model,model_nc)
        SSIM[j].append(a)
        PC[j].append(b)
        RMSE[j].append(c)
        PSNR[j].append(d)

In [ ]:
#from importlib import reload 
#import utils.causal_utils 
#reload(utils.causal_utils)
#from utils.causal_utils import testing
testing(np.array(SSIM),np.array(PC),np.array(RMSE),np.array(PSNR),test_loaders, Ao.to(device="cpu"),model, model_nc)